# Get Cocrystalized molecules from PDB

- Extract the cocrystalized molecules from the PDB structures.
- The active site is defined by a reference PDB structure and the name of the cocrystalized molecule (3-letter identifier code).

In [1]:
import itertools
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append('../..')
%load_ext autoreload
%autoreload 2

In [2]:
# Read the metadata dataframe created in the previous notebook
prot_name = 'cdk2'
df_prot = pd.read_pickle(f'./df_metadata_{prot_name}_pdbids.pkl')

In [3]:
df_prot_ligs = df_prot[df_prot.NumLigs > 0]
print(F'{len(df_prot_ligs)} of {len(df_prot)} structures have at least one HETATM molecule.')

391 of 418 structures have at least one HETATM molecule.


In [4]:
# How many cocrystalized molecules are there?
# A set of all HETATM molecules
lig_full_list = list( itertools.chain.from_iterable( df_prot.NameLigs ) )
print(f'There are a total of {len(lig_full_list)}', 
       'HETATM molecules (duplicates included).')

lig_all_list = list(set(lig_full_list))
print(f'There are {len(lig_all_list)} unique', 
       'HETATM molecules (without duplicates).')

There are a total of 538 HETATM molecules (duplicates included).
There are 359 unique HETATM molecules (without duplicates).


## Visualize pocket residues

In [5]:
from helper_modules.get_cocristalized_ligands \
     import get_pocket_ligand, PocketResidues

In [6]:
# Use a reference structure to visualize the pocket residues
ref_struc_id    = '1fin'
ref_ligand_name = 'ATP'

In [7]:
pkt_info = PocketResidues(ref_struc_id, ref_ligand_name)
pocket_residues_str = pkt_info.get_pocket_residues_as_list(cutoff = 8)
print('Indices of pocket redsidues:', pocket_residues_str)

Indices of pocket redsidues: 9 10 11 12 13 14 15 16 17 18 19 20 30 31 32 33 47 51 64 65 79 80 81 82 83 84 85 86 87 88 89 127 129 130 131 132 133 134 135 142 143 144 145 146 147 148


In [8]:
view = pkt_info.visualize_pocket()
view

NGLWidget()

In [9]:
view.render_image()
view._display_image()

## Perform the analysis
### Identify and save the molecules bound to the active site

- Define the input files and the output directory.

In [10]:
# Define the output directories
OUT_MAIN           = '../../data/crystal_confs/'
IN_DIR_PROT_CHAINS = f'{OUT_MAIN}/pdb_modeled'
IN_DIR_RAW_LIGS    = f'{OUT_MAIN}/raw_ligands'
OUT_DIR_PREP_LIGS  = f'{OUT_MAIN}/pocket_ligands'

# Create the output directory if it doesnt exists
Path(OUT_DIR_PREP_LIGS).mkdir(parents = True, exist_ok = True)

Update the **metadata DataFrame** with the information of the ligands:
- `LigPktName`: Name of the molecule found inside the pocket.
- `LigMW`: Molecular weight of the molecule (no Hydrogens).
- `LigNHeavyAtoms`: Number of heavy atoms of the molecule.

**Criteria** to identify a molecule as a <mark>ligand bound to the protein pocket</mark> (active site):
- The molecule has a `HETATM` identifier.
- The geometric center of the molecule is at most 10 Angstroms from the geometric center of the pocket. 
    - As an example, for `3pxf` the ligand `2AN` is 9.05 Angstroms from the pocket.
- The molecule has at least 12 heavy atoms (`min_heavy_atoms`). 
    - The molecule is not a solvent molecule.
- If two or more molecules satisfy the above criteria:
    - Select the molecule closest to the protein's pocket geometric center.
    
Additionally, only the coordinates of the molecule identified as ligand will be saved in the `OUT_DIR_PREP_LIGS` directory.

In [11]:
# Get all pocket ligands
lig_names, lig_mws, lig_n_atoms, lig_distances = [], [], [], []

for pdb_id in df_prot['PDB_ID']:
    name, mw, num_heavy_atoms, dist = get_pocket_ligand(
                        pdb_id          = pdb_id, 
                        pocket_residues = pocket_residues_str,
                        raw_lig_dir     = IN_DIR_RAW_LIGS,
                        prot_chain_dir  = IN_DIR_PROT_CHAINS,
                        pk_ligs_dir     = OUT_DIR_PREP_LIGS,
                        write_files     = True, 
                        min_heavy_atoms = 12,
                        min_dist_from_pkt = 10
                 )
    lig_names.append(name) 
    lig_mws.append(mw)
    lig_n_atoms.append(num_heavy_atoms)
    lig_distances.append(dist)
    
df_prot_lig = df_prot.copy(deep = True)
df_prot_lig["LigPktName"]      = lig_names
df_prot_lig["LigMW"]           = lig_mws
df_prot_lig["LigNHeavyAtoms"]  = lig_n_atoms
df_prot_lig["LigDistToPocket"] = lig_distances

Protein 1aq1
	Molecules found: ['STU']
	-> ligand STU saved (2.55 A from pkt center, 35 atoms)
Protein 1b38
	Molecules found: ['ATP', 'MG']
	-> ligand ATP saved (3.00 A from pkt center, 31 atoms)
Protein 1b39
	Molecules found: ['ATP', 'MG']
	-> ligand ATP saved (3.01 A from pkt center, 31 atoms)
Protein 1buh
	The protein 1buh has no ligand inside the pocket.
Protein 1ckp
	Molecules found: ['EDO', 'PVB']
	-> ligand PVB saved (4.83 A from pkt center, 18 atoms)
Protein 1di8
	Molecules found: ['DTQ']
	-> ligand DTQ saved (2.75 A from pkt center, 22 atoms)
Protein 1dm2
	Molecules found: ['EDO', 'HMD']
	-> ligand HMD saved (1.75 A from pkt center, 19 atoms)
Protein 1e1v
	Molecules found: ['CMG']
	-> ligand CMG saved (3.01 A from pkt center, 18 atoms)
Protein 1e1x
	Molecules found: ['NW1']
	-> ligand NW1 saved (3.06 A from pkt center, 18 atoms)
Protein 1e9h
	Molecules found: ['INR', 'TPO']
	-> ligand INR saved (2.41 A from pkt center, 24 atoms)
Protein 1fin
	Molecules found: ['ATP']
	-> ligan

	Molecules found: ['D31']
	-> ligand D31 saved (2.98 A from pkt center, 31 atoms)
Protein 2bhe
	Molecules found: ['BRY']
	-> ligand BRY saved (3.32 A from pkt center, 21 atoms)
Protein 2bhh
	Molecules found: ['RYU']
	-> ligand RYU saved (2.52 A from pkt center, 31 atoms)
Protein 2bkz
	Molecules found: ['SBC']
	-> ligand SBC saved (2.29 A from pkt center, 25 atoms)
Protein 2bpm
	Molecules found: ['529']
	-> ligand 529 saved (6.20 A from pkt center, 25 atoms)
Protein 2btr
	Molecules found: ['U73']
	-> ligand U73 saved (4.92 A from pkt center, 18 atoms)
Protein 2bts
	Molecules found: ['U32']
	-> ligand U32 saved (5.16 A from pkt center, 19 atoms)
Protein 2c4g
	Molecules found: ['514']
	-> ligand 514 saved (3.15 A from pkt center, 20 atoms)
Protein 2c5n
	Molecules found: ['CK8']
	-> ligand CK8 saved (2.65 A from pkt center, 23 atoms)
Protein 2c5o
	Molecules found: ['CK2']
	-> ligand CK2 saved (1.86 A from pkt center, 14 atoms)
Protein 2c5v
	Molecules found: ['CK4']
	-> ligand CK4 saved (2.

	-> ligand WXV saved (2.73 A from pkt center, 29 atoms)
Protein 2x1n
	Molecules found: ['X1N']
	-> ligand X1N saved (2.99 A from pkt center, 24 atoms)
Protein 2xmy
	Molecules found: ['CDK']
	-> ligand CDK saved (4.74 A from pkt center, 29 atoms)
Protein 2xnb
	Molecules found: ['Y8L']
	-> ligand Y8L saved (4.08 A from pkt center, 27 atoms)
Protein 3bht
	Molecules found: ['MFR', 'TPO']
	-> ligand MFR saved (1.94 A from pkt center, 18 atoms)
Protein 3bhu
	Molecules found: ['MHR', 'TPO']
	-> ligand MHR saved (1.56 A from pkt center, 20 atoms)
Protein 3bhv
	Molecules found: ['TPO', 'VAR']
	-> ligand VAR saved (1.39 A from pkt center, 22 atoms)
Protein 3ddp
	Molecules found: ['RC8', 'TPO']
	-> ligand RC8 saved (4.96 A from pkt center, 32 atoms)
Protein 3ddq
	Molecules found: ['RRC', 'TPO']
	-> ligand RRC saved (3.36 A from pkt center, 26 atoms)
Protein 3dog
	Molecules found: ['NNN', 'TPO']
	-> ligand NNN saved (3.16 A from pkt center, 26 atoms)
Protein 3eid
	Molecules found: ['PO5']
	-> liga

	Molecules found: ['O1Z']
	-> ligand O1Z saved (3.47 A from pkt center, 25 atoms)
Protein 3rai
	Molecules found: ['X85']
	-> ligand X85 saved (5.21 A from pkt center, 30 atoms)
Protein 3rak
	Molecules found: ['03Z']
	-> ligand 03Z saved (3.93 A from pkt center, 25 atoms)
Protein 3ral
	Molecules found: ['04Z']
	-> ligand 04Z saved (3.59 A from pkt center, 27 atoms)
Protein 3rjc
	Molecules found: ['06Z']
	-> ligand 06Z saved (3.54 A from pkt center, 22 atoms)
Protein 3rk5
	Molecules found: ['07Z']
	-> ligand 07Z saved (3.77 A from pkt center, 24 atoms)
Protein 3rk7
	Molecules found: ['08Z']
	-> ligand 08Z saved (3.60 A from pkt center, 24 atoms)
Protein 3rk9
	Molecules found: ['09Z']
	-> ligand 09Z saved (2.30 A from pkt center, 18 atoms)
Protein 3rkb
	Molecules found: ['12Z']
	-> ligand 12Z saved (3.02 A from pkt center, 21 atoms)
Protein 3rm6
	Molecules found: ['18Z']
	-> ligand 18Z saved (3.49 A from pkt center, 30 atoms)
Protein 3rm7
	Molecules found: ['19Z']
	-> ligand 19Z saved (4.

Protein 4gcj
	Molecules found: ['EDO', 'X64']
	-> ligand X64 saved (3.28 A from pkt center, 28 atoms)
Protein 4i3z
	Molecules found: ['ADP', 'CL', 'GOL', 'MG', 'TPO']
	-> ligand ADP saved (1.43 A from pkt center, 27 atoms)
Protein 4ii5
	Molecules found: ['ADP', 'GOL', 'MG', 'TPO']
	-> ligand ADP saved (1.56 A from pkt center, 27 atoms)
Protein 4kd1
	Molecules found: ['1QK', 'EDO']
	-> ligand 1QK saved (4.28 A from pkt center, 29 atoms)
Protein 4lyn
	Molecules found: ['1YG']
	-> ligand 1YG saved (3.49 A from pkt center, 27 atoms)
Protein 4nj3
	Molecules found: ['2KD']
	-> ligand 2KD saved (3.96 A from pkt center, 36 atoms)
Protein 4rj3
	Molecules found: ['3QS', 'ACT', 'ALY']
	-> ligand 3QS saved (3.20 A from pkt center, 29 atoms)
Protein 5a14
	Molecules found: ['LQ5']
	-> ligand LQ5 saved (6.44 A from pkt center, 36 atoms)
Protein 5and
	Molecules found: ['5JE']
	-> ligand 5JE saved (4.85 A from pkt center, 14 atoms)
Protein 5ane
	Molecules found: ['SZL']
	XXX The protein 5ane has no lig

## Save the updated DataFrame

In [19]:
file = f'./df_metadata_{prot_name}_pdbids_LIGS.pkl'
df_prot_lig = df_prot_lig.reset_index().drop('index', axis = 1)
df_prot_lig.to_pickle(file)
df_prot_lig.sample(10)

,PDB_ID,Title,Date,Entities,ChainID,Resolution,Identity,Coverage,NumGaps,GapLen,GapPos,NumLigs,NameLigs,NonStndResnames,NonStndResnums,LigPktName,LigMW,LigNHeavyAtoms,LigDistToPocket
300,4bcq,comparative structural and functional studies ...,2013-01-09,"1,2,3",A,2.4,0.993,97.315,2,"[5, 3]","[[11, 15], [296, 298]]",1,[TJF],[TPO],[160],TJF,414.335400,31.0,2.832216
28,1h1r,structure-based design of a potent purine-base...,2002-09-19,"1,2",A,2.0,0.993,99.329,1,[2],"[[296, 297]]",1,[6CP],[TPO],[160],6CP,337.678500,25.0,2.979890
70,1qmz,the structural basis for specificity of substr...,1999-12-14,"1,2,3",A,2.2,0.993,99.329,1,[2],"[[296, 297]]",2,"[ATP, MG]",[TPO],[160],ATP,491.053983,31.0,2.130757
301,4bgh,the lab oddity prevails: discovery of pan-cdk ...,2013-09-04,1,A,1.95,0.993,92.282,2,"[9, 14]","[[36, 44], [150, 163]]",1,[3I6],None,None,3I6,386.139800,23.0,4.583283
185,3eid,"synthesis and evaluation of pyrazolo[1,5-b]pyr...",2008-10-21,"1,2",A,3.15,0.993,97.987,3,"[3, 2, 1]","[[38, 40], [161, 162], [298, 298]]",1,[PO5],None,None,PO5,460.319300,36.0,2.794145
110,2cch,the role of the phospho-cdk2/cyclin a recruitm...,2006-05-03,"1,2,3",A,1.7,0.993,99.329,1,[2],"[[296, 297]]",3,"[ATP, GOL, SO4]",[TPO],[160],ATP,491.053983,31.0,3.684462
379,5uq1,structural basis of divergent cyclin-dependent...,2017-07-05,"1,2",A,3.2,0.993,96.644,3,"[2, 6, 2]","[[1, 2], [22, 27], [296, 297]]",0,[],None,None,None,NaN,NaN,NaN
55,1ol2,insights into cyclin groove recognition. compl...,2003-12-11,"1,2,3",A,2.6,0.993,99.329,1,[2],"[[296, 297]]",0,[],None,None,None,NaN,NaN,NaN
368,5jq8,inhibitor selectivity for cyclin-dependent kin...,2017-02-08,1,A,1.94,0.993,93.289,3,"[2, 9, 9]","[[1, 2], [36, 44], [153, 161]]",1,[I73],None,None,I73,364.275000,29.0,3.654302
199,3ns9,"a novel pyrazolo[1,5-a]pyrimidine is a potent ...",2010-12-08,1,A,1.78,0.993,96.98,2,"[2, 7]","[[1, 2], [36, 42]]",1,[NS9],None,None,NS9,358.245700,28.0,3.878558


In [13]:
df_prot_lig.shape

(418, 19)